# Introduction
This notebook shows how to connect to a crypto exchange and receive a trade feed from it. We then start the run using this trade feed.  Besides the support for Binance (see also the other notebook), roboquant includes support for most other crypto exchanges using the XChange library. This notebook demonstrates how to use that library.

> XChange is a Java library providing a streamlined API for interacting with 60+ Bitcoin and Altcoin exchanges providing a consistent interface for trading and accessing market data. 


In [ ]:
%use roboquant(1.0.0-SNAPSHOT)
@file:DependsOn("org.roboquant:roboquant-crypto:1.0.0-SNAPSHOT")

import org.roboquant.xchange.*
Welcome()

# Add required libraries
When using the XChange integration, there are a few additional steps required to use a particular exchange:

1. Load the exchange specific libraries, like bitstamp in this example. You can use the **@file:DependsOn** syntax for this. Make sure the version is the same as is included with roboquant.
2. Import both the roboquant packages (import org.roboquant.xchange.*) and the packages for the exchange that you want to connect to.

Check the [XChange](https://github.com/knowm/XChange) repository for more details on all the available exchanges. In this example we use Bitstamp

In [ ]:
@file:DependsOn("org.knowm.xchange:xchange-bitstamp:5.0.13")
@file:DependsOn("org.knowm.xchange:xchange-stream-bitstamp:5.0.13")

import info.bitrich.xchangestream.bitstamp.v2.BitstampStreamingExchange
import info.bitrich.xchangestream.core.StreamingExchangeFactory

# Setup a Roboquant
We now create, as usual, the strategy we want to test. There is nothing specific to crypto trading that needs to be taken care of.

In [ ]:
val strategy = EMAStrategy.PERIODS_5_15
val roboquant =  Roboquant(strategy, AccountMetric())

# Define Feed
Now we create an instance of the Bitstamp exchange and connect to it. Then we are ready to use it as a feed and subscribe to one or more currency pairs using the *subscribeTicker* method.

In [ ]:
val exchange = StreamingExchangeFactory.INSTANCE.createExchange(BitstampStreamingExchange::class.java)
exchange.connect().blockingAwait()

val feed = XChangeLiveFeed(exchange)

In [ ]:
feed.subscribeTicker("BTC_USD")

# Run Live Test
All that remains, is to start run to roboquant and evaluate the strategy against the feed. We'll run it for 10 minutes. If the time that is displayed looks wrong, don't forget that roboquant uses a timezone independent representation for all internal time processing and not your local timezone.

In [ ]:
val timeframe = Timeframe.next(60.minutes)
roboquant.run(feed, timeframe)

# Disconnect
And we disconnect from the exchange

In [ ]:
exchange.disconnect().blockingAwait()

In [ ]:
roboquant.broker.account.fullSummary()